# Генерация синтетических данных на основе выданных
Сюда приходит файл __X_train_clothes.csv__, содержащий отзывы только из категории "одежда"
Остальные файлы используются лишь для как различного рода проверок (__X_train.csv__)

In [1]:
!pip uninstall -y torch torchvision torchaudio fastai
!pip install -q "transformers==4.41.2" "datasets==2.19.1" "accelerate==0.30.1" "bitsandbytes==0.43.1" "torch==2.3.0"

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: fastai 2.7.19
Uninstalling fastai-2.7.19:
  Successfully uninstalled fastai-2.7.19
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 75.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 MB 2

## Подготовка к генерации (Загрузка данных и модели)

In [85]:
X_train_clothes = pd.read_csv('/kaggle/input/traint-bank-sirius/X_train_clothes.csv')

In [37]:
import pandas as pd
categories = [
    'бытовая техника',
    'обувь',
    'одежда',
    'посуда',
    'текстиль',
    'товары для детей',
    'украшения и аксессуары',
    'электроника',
    'нет товара'
]

X_train = pd.read_csv('/kaggle/input/traint-bank-sirius/X_train.csv')
y_train = pd.read_csv('/kaggle/input/traint-bank-sirius/y_true.csv', header=None, names=['target'])
df_qwen_v1 = pd.read_csv('/kaggle/input/traint-bank-sirius/qwen2.5-7B_instruct_df_train_v1.csv')
y_qwen_v2 = pd.read_csv('/kaggle/input/traint-bank-sirius/predictions_qwen_v2.csv')
# df_qwen_v1['pred']

In [39]:
X_train_full = X_train.copy()
X_train_full['target'] = y_train
X_train_full['pred_qwen_v2'] = y_qwen_v2
X_train_full[(X_train_full['target'] == 'одежда') & (X_train_full['pred_qwen_v2'] == 'одежда')]
X_train_full['text'].to_csv('/kaggle/working/X_train_clothes_v1.csv')

In [6]:
y_qwen_v2['text'][0] = 'одежда'

/tmp/ipykernel_36/1151428347.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  y_qwen_v2['text'][0] = 'одежда'


In [7]:
mask_neq = (y_train['target'] != df_qwen_v1['pred'])
X_train_all = X_train[mask_neq].copy()
X_train_all['target'] = y_train[mask_neq]['target']
X_train_all['pred'] = df_qwen_v1[mask_neq]['pred']
pd.set_option('display.max_rows', False)
X_train_all.to_csv('X_train_all.csv')

In [59]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "Qwen/Qwen2.5-7B-Instruct"

# Задаем конфиг для 4-битной квантизации
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

#загружаем модель с квантизацией
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16
)
print("модель и токенизатор загружены")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

модель и токенизатор загружены


## Генерация данных на основе выданных

#### Как вообще будет происходить генерация.
Просмотрев датасет, я выяснил у него есть только два достаточно полных класса - "нет товара" и "одежда". Очевидно, что класс "нет товара" существенно отличается от других краткостью, неинформативностью отзывов, поэтому отзывы из этой категории будут нерелевантны для генерации отзывов других категорий ('бытовая техника','обувь', 'посуда','текстиль', 'товары для детей','украшения и аксессуары','электроника').

А вот класс "одежда" отлично подойдет для этой задачи. Поэтому мы будем генерировать данные, отталкиваясь только от этого класса.

Генерировать будем следующим образом.
Параметры генерации.
1) Есть название товара
   * В каком месте название товара в отзыве (начало/середина/конец)
2) Есть только описание товара
   * В каком месте описание товара в отзыве (начало/середина/конец)
По хорошему нужно было помимо разметки данных, сохранять информации о размышлении модели(на основании чего была выбрана та или иная категория (название товара, описание)), чтобы для первого типа генерации подавать примеры с таким же типом из другого класса, для второго типа - ... .Но что уже поделать, не знал, что это потребуется, а время на это - нет(((

Как устроен промпт

на вход __build_prompt__ подаваться
*   Категория для которой нужно сгенерировать пример.
*   Название самого товара (`product_name`).
*   Тип генерации (`generation_type`): 'название' или 'описание'.
*   Позиция в тексте (`position`): 'начало', 'середина', 'конец'.
*   **Тональность отзыва (`sentiment`): 'позитивный', 'негативный'.**
*   **Пол автора (`gender`): 'мужчина', 'женщина'.**
*   10 примеров из категории 'одежда' (`examples`).
*   Часть описания выбора категории (`classifier_prompt_part`).

Это описание я писал давно, так что возможно стоит взглянуть на код

In [72]:
# Список названий товаров для категорий для которых нужно сгенерировать данные
PRODUCT_NAME_STORAGE = {
    "товары для детей": [
        "игрушка", "конструктор", "кукла", "машинка", "коляска", "соска", "подгузник",
        "кроватка", "автокресло", "самокат", "пинетки", "боди", "ползунки", "детский комбинезон",
        "стульчик для кормления", "манеж", "погремушка", "бутылочка для кормления", "слюнявчик",
        "развивающий коврик", "мобиль на кроватку", "ходунки", "прыгунки", "велосипед трехколесный",
        "беговел", "детский шампунь", "влажные салфетки", "ванночка для купания", "горшок",
        "радионяня", "видеоняня", "сортер", "пирамидка", "детские пазлы", "книжка-игрушка",
        "детское мыло", "термометр для воды", "пустышка", "прорезыватель для зубов", "царапки",
        "чепчик", "детский крем", "присыпка", "ниблер", "стерилизатор для бутылочек"
    ],
    "обувь": [
        "кроссовки", "кеды", "туфли", "ботинки", "сапоги", "босоножки", "сандалии",
        "шлепанцы", "тапочки", "ботильоны", "балетки", "угги", "мокасины", "слипоны",
        "лоферы", "эспадрильи", "сабо", "мюли", "ботфорты", "полусапоги", "резиновые сапоги",
        "дутики", "валенки", "унты", "бутсы", "чешки", "аквашузы", "ортопедическая обувь",
        "рабочие ботинки", "берцы", "гриндерсы", "оксфорды", "броги", "топсайдеры",
        "домашние тапочки", "вьетнамки", "сланцы", "кроксы", "хайтопы", "треккинговые ботинки"
    ],
    "украшения и аксессуары": [
        "сумка", "рюкзак", "клатч", "кошелек", "шапка", "шляпа", "кепка", "шарф", "платок",
        "перчатки", "ремень", "очки", "часы", "зонт", "кольцо", "серьги", "браслет", "цепочка",
        "колье", "брошь", "галстук", "панама", "берет", "повязка на голову", "бейсболка",
        "снуд", "палантин", "варежки", "митенки", "сумка-шоппер", "поясная сумка", "косметичка",
        "визитница", "ключница", "кулон", "подвеска", "запонки", "зажим для галстука",
        "пирсинг", "кафф", "заколка", "резинка для волос", "ободок", "галстук-бабочка",
        "портмоне", "обложка для паспорта", "бандана", "солнцезащитные очки", "кардхолдер"
    ],
    "текстиль": [
        "постельное белье", "простыня", "пододеяльник", "наволочка", "одеяло", "подушка",
        "плед", "покрывало", "полотенце", "скатерть", "шторы", "тюль", "чехол на диван",
        "наматрасник", "декоративная наволочка", "банный халат", "коврик для ванной",
        "кухонное полотенце", "прихватка", "фартук", "салфетка тканевая", "дорожка на стол",
        "чехол на кресло", "чехол на стул", "римская штора", "портьера", "полотенце для рук",
        "полотенце для ног", "пляжное полотенце", "гобелен", "флисовый плед", "махровая простыня",
        "ортопедическая подушка", "подушка для беременных", "одеяло из верблюжьей шерсти",
        "шелковое постельное белье", "льняная скатерть", "вафельное полотенце"
    ],
    "посуда": [
        "тарелка", "миска", "чашка", "кружка", "стакан", "бокал", "вилка", "ложка", "нож",
        "кастрюля", "сковорода", "противень", "форма для выпечки", "доска разделочная",
        "чайник (неэлектрический)", "блюдце", "салатник", "пиала", "супница", "блюдо для сервировки",
        "менажница", "соусник", "масленка", "сахарница", "солонка", "перечница", "салфетница",
        "поднос", "рюмка", "стопка", "фужер", "графин", "кувшин", "сотейник", "ковш", "дуршлаг",
        "сито", "половник", "шумовка", "венчик", "скалка", "терка", "овощечистка", "штопор",
        "открывалка", "мерный стакан", "контейнер для еды", "ланч-бокс", "термос", "френч-пресс"
    ],
    "бытовая техника": [
        "холодильник", "плита", "духовка", "микроволновка", "чайник электрический", "кофеварка",
        "миксер", "блендер", "тостер", "мультиварка", "стиральная машина", "пылесос", "утюг",
        "фен", "обогреватель", "вентилятор", "эпилятор", "электробритва", "посудомоечная машина",
        "морозильная камера", "кухонная вытяжка", "соковыжималка", "мясорубка", "кухонные весы",
        "йогуртница", "вафельница", "гриль электрический", "хлебопечка", "пароварка", "сушильная машина",
        "робот-пылесос", "пароочиститель", "отпариватель", "увлажнитель воздуха", "осушитель воздуха",
        "ионизатор", "швейная машина", "электрическая зубная щетка", "ирригатор", "массажер",
        "весы напольные", "машинка для стрижки волос", "триммер", "кондиционер", "тепловентилятор"
    ],
    "электроника": [
        "телефон", "смартфон", "планшет", "ноутбук", "наушники", "колонка", "телевизор",
        "монитор", "фотоаппарат", "зарядное устройство", "пауэрбанк", "флешка", "карта памяти",
        "кабель", "мышь", "клавиатура", "смарт-часы", "фитнес-браслет", "роутер", "веб-камера",
        "принтер", "сканер", "проектор", "внешний жесткий диск", "SSD-накопитель", "саундбар",
        "домашний кинотеатр", "видеокамера", "экшн-камера", "диктофон", "электронная книга",
        "игровая приставка", "геймпад", "джойстик", "игровой руль", "умная колонка", "умная розетка",
        "умная лампочка", "квадрокоптер", "дрон", "GPS-навигатор", "видеорегистратор",
        "Wi-Fi репитер", "графический планшет", "микрофон", "процессор", "видеокарта"
    ]
}

In [77]:
# Когда я делал разметку датасета X_train.csv у меня сформировалась инструкция, которая 
# теперь потребуется для генерации отзывов миноритарных классов

#Благодаря такому разбиению промпт не будет содержать информацию о всех классах, 
# что очень удобно, потому что для больших промптов результаты могут быть плачевными)


PROMPT_PARTS = {
    # Этот блок будет использоваться в КАЖДОМ вызове.
    "general_rules": """
--- ГЛАВНЫЕ ПРИНЦИПЫ КЛАССИФИКАЦИИ (для использования на ШАГЕ 1)--- 
**ПРИНЦИП №1: УПОМИНАНИЕ ТОВАРА ВАЖНЕЕ ВСЕГО.**
Если в отзыве прямо назван конкретный товар (например, 'трусы', 'платье', 'телефон'), ты ОБЯЗАН выбрать категорию этого товара, даже если вся остальная часть отзыва посвящена проблемам с доставкой или спором.
-   **КЛЮЧЕВОЙ ПРИМЕР:** "Трусы не пришли"
-   **РАЗМЫШЛЕНИЯ:** Товар 'трусы' четко назван. 'Трусы' относятся к категории 'одежда'. Жалоба "не пришли" касается доставки, но так как товар назван, мы игнорируем проблему доставки и классифицируем по товару.
-   **РЕШЕНИЕ:** одежда

--- АЛГОРИТМ ПРИНЯТИЯ РЕШЕНИЯ ---
**ШАГ 1: ПРОВЕРКА НА ПРЯМОЕ УПОМИНАНИЕ ТОВАРА (ЗОЛОТОЕ ПРАВИЛО).**
Сначала найди в отзыве название конкретного товара.
-   **ЕСЛИ** в отзыве назван товар (например, 'рюкзак', 'майка', 'трусы', 'платье'), **ТО** немедленно определи его категорию и **ПОЛНОСТЬЮ ПРОИГНОРИРУЙ** все остальные слова про доставку, споры или деньги.

**ШАГ 2: ПРОВЕРКА НА ФИЗИЧЕСКОЕ ОПИСАНИЕ.**
-   **ЕСЛИ** на ШАГЕ 1 товар не был найден, проверь, есть ли в отзыве физическое описание или дефект ('низ рваный', 'ткань колется', 'не включается').
-   **ЕСЛИ** описание есть, определи категорию по нему.

**ШАГ 3: КАТЕГОРИЯ "НЕТ ТОВАРА".**
-   **ЕСЛИ** ШАГ 1 и ШАГ 2 НЕ ПРОЙДЕНЫ, **ТОЛЬКО ТОГДА** выбери категорию **'нет товара'**.

--- ГЛАВНЫЕ ПРИНЦИПЫ КЛАССИФИКАЦИИ (для использования на ШАГЕ 2)--- 
**ПРИНЦИП №1: ФИЗИЧЕСКОЕ ОПИСАНИЕ ВАЖНЕЕ ЖАЛОБ НА СЕРВИС.**
Если товар не назван, но есть его физическое описание или дефект ('низ рваный', 'ткань колется', 'не включается'), классифицируй по этому описанию. Жалобы на споры или деньги в этом случае игнорируются.
-   **ПРИМЕР:** "Низ рваный..деньги не вернули! Открыла спор." -> РЕШЕНИЕ: одежда
-   **[НЕГАТИВНЫЙ ПРИМЕР]** "Ужасная синтетика" -> РЕШЕНИЕ: нет товара
""",

    # Этот словарь содержит ТОЛЬКО специфичную для категории информацию.
    "category_specific_rules": {
        "товары для детей": """**КАТЕГОРИЯ: товары для детей (ВЫСШИЙ ПРИОРИТЕТ)**
**ПРАВИЛО:** Любой товар, если он явно предназначен для ребенка.
**КЛЮЧЕВЫЕ СЛОВА И ТЕМЫ:** игрушка, конструктор, кукла, 'сыну'/'дочке', 'ребенок доволен'.
-   **Пример:** "Куртка для сына оказалась очень теплой."
-   **РЕШЕНИЕ:** товары для детей.""",

        "нет товара": """**КАТЕГОРИЯ: нет товара**
**ПРАВИЛО:** Выбирай эту категорию ТОЛЬКО если из отзыва абсолютно невозможно понять, о каком товаре идет речь.
**СЮДА ОТНОСИТСЯ:** Отзывы только о доставке, упаковке, продавце, возврате денег.
-   **Пример:** "Шло очень долго, но упаковано хорошо."
-   **РЕШЕНИЕ:** нет товара.""",

        "одежда": """**КАТЕГОРИЯ: одежда**
**ПРАВИЛО:** Вещи, которые надевают на тело. ИСКЛЮЧАЕТ обувь и аксессуары.
**КЛЮЧЕВЫЕ СЛОВА И ТЕМЫ:** платье, юбка, брюки, джинсы, кофта; размер (S, M, 44), 'село идеально', 'маломерит', 'рукава короткие'.
-   **Пример:** "Кофта хорошая, теплая, но после стирки вся в катышках."
-   **РЕШЕНИЕ:** одежда.""",

        "обувь": """**КАТЕГОРИЯ: обувь**
**ПРАВИЛО:** Все, что имеет подошву и надевается на стопу.
**КЛЮЧЕВЫЕ СЛОВА И ТЕМЫ:** кроссовки, туфли, ботинки; размер (39, 42), 'жмут', 'натирают пятку', 'удобная колодка'.
-   **Пример:** "На свой 38 взяла 39, и все равно жмут. Очень узкие."
-   **РЕШЕНИЕ:** обувь.""",

        "украшения и аксессуары": """**КАТЕГОРИЯ: украшения и аксессуары**
**ПРАВИЛО:** Предметы для украшения и дополнения образа. **Сюда относятся шапки, шарфы, перчатки, сумки, ремни.**
**КЛЮЧЕВЫЕ СЛОВА И ТЕМЫ:** серьги, кольцо, браслет, сумка; 'застежка', 'камень выпал', 'потемнело', 'молния сломалась'.
-   **Пример:** "Сумка красивая, но фурнитура очень дешевая, вся облезла."
-   **РЕШЕНИЕ:** украшения и аксессуары.""",
        
        "текстиль": """**КАТЕГОРИЯ: текстиль**
**ПРАВИЛО:** Тканевые изделия для дома и интерьера. Не одежда.
**КЛЮЧЕВЫЕ СЛОВА И ТЕМЫ:** постельное белье, простыня, полотенце, шторы; 'мягкое', 'впитывает воду', 'после стирки село', 'полиняло'.
-   **Пример:** "Полотенца ужасные, жесткие и совсем не впитывают воду."
-   **РЕШЕНИЕ:** текстиль.""",

        "посуда": """**КАТЕГОРИЯ: посуда**
**ПРАВИЛО:** Предметы для готовки и еды, не являющиеся электроприборами.
**КЛЮЧЕВЫЕ СЛОВА И ТЕМЫ:** кастрюля, сковорода, тарелка, чашка; 'пригорает', 'ручка греется', 'скол', 'треснула'.
-   **Пример:** "На сковородке все пригорает, покрытие облезло."
-   **РЕШЕНИЕ:** посуда.""",

        "бытовая техника": """**КАТЕГОРИЯ: бытовая техника**
**ПРАВИЛО:** Электрические приборы для дома (не персональные гаджеты).
**КЛЮЧЕВЫЕ СЛОВА И ТЕМЫ:** пылесос, утюг, чайник, миксер; 'не работает', 'шумит', 'греется', 'мощность', 'кнопка'.
-   **Пример:** "Чайник воняет пластиком, даже после десятого кипячения."
-   **РЕШЕНИЕ:** бытовая техника.""",

        "электроника": """**КАТЕГОРИЯ: электроника**
**ПРАВИЛО:** Персональные цифровые устройства, гаджеты и комплектующие.
**КЛЮЧЕВЫЕ СЛОВА И ТЕМЫ:** телефон, наушники, планшет, смарт-часы; 'батарея', 'экран', 'звук', 'bluetooth', 'не подключается', 'зависает'.
-   **Пример:** "Беспроводные наушники постоянно теряют связь с телефоном."
-   **РЕШЕНИЕ:** электроника."""
    }
}

In [50]:
import pandas as pd
import json
from typing import List, Dict
from itertools import cycle

#Создаем промпт для генерации отзывов
def build_generation_prompt(
    product_name: str,
    target_category: str,
    style_examples_df: pd.DataFrame,
    general_rules: str,
    category_specific_rules: str
) -> str:
    # ... (код этой функции остается без изменений)
    examples_list = style_examples_df['text'].tolist()
    examples_str = "\n".join([f"- {ex}" for ex in examples_list])
    focused_instructions = f"{general_rules}\n\n{category_specific_rules}"
    master_prompt = f"""
Твоя задача — создать набор реалистичных отзывов для товара.

--- ТЕХНИЧЕСКОЕ ЗАДАНИЕ (ПРАВИЛА КЛАССИФИКАЦИИ) ---
Внимательно изучи эти правила. Твои сгенерированные отзывы должны стать идеальными примерами для этих правил.
{focused_instructions}

--- ПРИМЕРЫ СТИЛЯ ---
Твои отзывы должны быть похожи на них по стилю.
{examples_str}

--- ОСНОВНАЯ ЗАДАЧА: СИСТЕМАТИЧЕСКАЯ ГЕНЕРАЦИЯ ---
Для товара "{product_name}" (категория "{target_category}") сгенерируй набор отзывов, покрывающих следующие случаи:
1.  Позитивный отзыв с явным упоминанием товара.
2.  Негативный отзыв с явным упоминанием товара и жалобой на качество.
3.  Отзыв с упоминанием товара, но основной жалобой на сервис (доставка/возврат).
4.  Отзыв с физическим описанием дефекта, но без упоминания названия товара.
5.  Отзыв, который должен быть классифицирован как 'нет товара'.

--- ВЫХОДНОЙ ФОРМАТ ---
Верни результат в виде JSON ОБЪЕКТА с ключом "reviews" (список объектов с ключами "text" и "target").
"""
    return master_prompt


# Функция для генерации одного товара
def generate_for_one_product(
    product_name: str,
    target_category: str,
    full_clothing_df: pd.DataFrame,
    prompt_parts: Dict
) -> pd.DataFrame:
    # ... (код этой функции остается без изменений)
    sample_size = min(15, len(full_clothing_df))
    sampled_examples_df = full_clothing_df.sample(n=sample_size)
    general_rules = prompt_parts['general_rules']
    category_specific_rules = prompt_parts['category_specific_rules'][target_category]
    final_prompt = build_generation_prompt(
        product_name=product_name,
        target_category=target_category,
        style_examples_df=sampled_examples_df,
        general_rules=general_rules,
        category_specific_rules=category_specific_rules
    )
    mock_response = f"""
    {{"reviews": [{{"text": "Отличная {product_name}, рекомендую!", "target": "{target_category}"}}, {{"text": "{product_name.capitalize()} пришла с царапиной.", "target": "{target_category}"}}, {{"text": "Мою {product_name} так и не доставили.", "target": "{target_category}"}}, {{"text": "Качество ужасное, сломалась через неделю.", "target": "{target_category}"}}, {{"text": "Доставили быстро, упаковка целая.", "target": "нет товара"}}]}}
    """
    llm_response_str = mock_response
    try:
        data = json.loads(llm_response_str)
        review_list = data["reviews"]
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Ошибка обработки ответа модели для '{product_name}': {e}")
        return pd.DataFrame(columns=['text', 'target'])
    return pd.DataFrame(review_list)


# Генерация датасета с нужным количеством объектов
#Генерируем до тех пор, пока не будет создано нужное количество примеров.
def generate_dataset(
    target_total_count: int,
    full_clothing_df: pd.DataFrame,
    categories_to_generate: List[str],
    product_storage: Dict,
    prompt_parts: Dict
) -> pd.DataFrame:
    
    all_generated_dfs = []
    total_generated_count = 0

    # 1. Создаем плоский список всех товаров, по которым будем циклично проходить.
    #    Формат: [('товар1', 'категория1'), ('товар2', 'категория1'), ...]
    product_cycle_list = []
    for category in categories_to_generate:
        products_in_category = product_storage.get(category, [])
        if products_in_category:
            for product in products_in_category:
                product_cycle_list.append((product, category))
        else:
            print(f"Предупреждение: для категории '{category}' нет товаров в хранилище.")

    if not product_cycle_list:
        print("Ошибка: нет товаров для генерации. Проверьте 'categories_to_generate' и 'product_storage'.")
        return pd.DataFrame()

    #бесконечный итератор для циклического прохода по товарам
    product_iterator = cycle(product_cycle_list)

    #работаем, пока не наберем нужное количество
    while total_generated_count < target_total_count:
        current_product, current_category = next(product_iterator)
        
        print(f"Генерация для: '{current_product}' ({current_category})... ", end="")
        
        product_df = generate_for_one_product(
            product_name=current_product,
            target_category=current_category,
            full_clothing_df=full_clothing_df,
            prompt_parts=prompt_parts
        )
        
        all_generated_dfs.append(product_df)
        total_generated_count += len(product_df)
        print(f"Добавлено {len(product_df)} примеров. Всего: {total_generated_count}/{target_total_count}")

    final_df = pd.concat(all_generated_dfs, ignore_index=True)
    
    # Обрезаем DataFrame до ровно 700 записей
    final_df = final_df.head(target_total_count)
    
    print(f"\n===== Генерация завершена. Итоговый размер датасета: {final_df.shape} =====")
    return final_df


Цель: сгенерировать 5 примеров.
Генерация для: 'Сковорода' (посуда)... Добавлено 5 примеров. Всего: 5/5

===== Генерация завершена. Итоговый размер датасета: (5, 2) =====

--- ИТОГОВЫЙ СГЕНЕРИРОВАННЫЙ DATAFRAME ---
                                        text      target
0            Отличная Сковорода, рекомендую!      посуда
1              Сковорода пришла с царапиной.      посуда
2          Мою Сковорода так и не доставили.      посуда
3  Качество ужасное, сломалась через неделю.      посуда
4          Доставили быстро, упаковка целая.  нет товара
...
                                        text      target
0            Отличная Сковорода, рекомендую!      посуда
1              Сковорода пришла с царапиной.      посуда
2          Мою Сковорода так и не доставили.      посуда
3  Качество ужасное, сломалась через неделю.      посуда
4          Доставили быстро, упаковка целая.  нет товара


In [69]:
def get_model_response(prompt: str, model, tokenizer, device: str) -> str:
    """
    Отправляет промпт в модель и возвращает очищенный текстовый ответ.
    """
    messages = [
        {"role": "system", "content": "You are a helpful assistant that generates data in JSON format."},
        {"role": "user", "content": prompt}
    ]
    inputs = tokenizer.apply_chat_template(
        messages, 
        tokenize=True, 
        add_generation_prompt=True, 
        return_tensors="pt"
    ).to(device)

    outputs = model.generate(
        inputs,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )

    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    json_start_index = full_response.find('{')
    
    if json_start_index != -1:
        potential_json = full_response[json_start_index:]
        json_end_index = potential_json.rfind('}')
        if json_end_index != -1:
            return potential_json[:json_end_index + 1]
        return potential_json # Возвращаем то, что нашли
    else:
        return '{"reviews": []}'



#### Генерация отзывов из категории __посуда__

In [ ]:
TARGET_COUNT = 500
CATEGORIES_TO_PROCESS = ["посуда"]
dataset_dishes = generate_dataset(
    target_total_count=TARGET_COUNT,
    full_clothing_df=X_train_clothes,
    categories_to_generate=CATEGORIES_TO_PROCESS,
    product_storage=PRODUCT_NAME_STORAGE,
    prompt_parts=PROMPT_PARTS,
    model=model,
    tokenizer=tokenizer,
    device=device
)
print(dataset_dishes)

Цель: сгенерировать 500 примеров.
Генерация для: 'тарелка' (посуда)... Добавлено 5 примеров. Всего: 5/500
Генерация для: 'миска' (посуда)... Добавлено 5 примеров. Всего: 10/500
Генерация для: 'чашка' (посуда)... Добавлено 5 примеров. Всего: 15/500
Генерация для: 'кружка' (посуда)... Добавлено 5 примеров. Всего: 20/500
Генерация для: 'стакан' (посуда)... Добавлено 5 примеров. Всего: 25/500
Генерация для: 'бокал' (посуда)... Добавлено 5 примеров. Всего: 30/500
Генерация для: 'вилка' (посуда)... Добавлено 5 примеров. Всего: 35/500
Генерация для: 'ложка' (посуда)... 
Ошибка обработки ответа модели для 'ложка': Expecting ',' delimiter: line 21 column 13 (char 791)
Добавлено 0 примеров. Всего: 35/500
Генерация для: 'нож' (посуда)... Добавлено 5 примеров. Всего: 40/500
Генерация для: 'кастрюля' (посуда)... Добавлено 5 примеров. Всего: 45/500
Генерация для: 'сковорода' (посуда)... Добавлено 5 примеров. Всего: 50/500
Генерация для: 'противень' (посуда)... 
Ошибка обработки ответа модели для 'пр

In [ ]:
dataset_dishes

In [ ]:
dataset_dishes.to_csv('dataset_dishes.csv')

#### Генерация отзывов из категории __бытовая техника__

In [ ]:
TARGET_COUNT = 500
CATEGORIES_TO_PROCESS = ["бытовая техника"]
dataset_house_apps = generate_dataset(
    target_total_count=TARGET_COUNT,
    full_clothing_df=X_train_clothes,
    categories_to_generate=CATEGORIES_TO_PROCESS,
    product_storage=PRODUCT_NAME_STORAGE,
    prompt_parts=PROMPT_PARTS,
    model=model,
    tokenizer=tokenizer,
    device=device
)
print(dataset_house_apps)

In [ ]:
dataset_house_apps.to_csv('dataset_house_apps.csv')

#### Генерация отзывов из категории __электроника__

In [ ]:
TARGET_COUNT = 500
CATEGORIES_TO_PROCESS = ["электроника"]
dataset_electronics = generate_dataset(
    target_total_count=TARGET_COUNT,
    full_clothing_df=X_train_clothes,
    categories_to_generate=CATEGORIES_TO_PROCESS,
    product_storage=PRODUCT_NAME_STORAGE,
    prompt_parts=PROMPT_PARTS,
    model=model,
    tokenizer=tokenizer,
    device=device
)
print(dataset_electronics)

In [ ]:
dataset_electronics.to_csv('dataset_electronics.csv')

#### Генерация отзывов из категории __текстиль__

In [ ]:
TARGET_COUNT = 500
CATEGORIES_TO_PROCESS = ["текстиль"]
dataset_textile = generate_dataset(
    target_total_count=TARGET_COUNT,
    full_clothing_df=X_train_clothes,
    categories_to_generate=CATEGORIES_TO_PROCESS,
    product_storage=PRODUCT_NAME_STORAGE,
    prompt_parts=PROMPT_PARTS,
    model=model,
    tokenizer=tokenizer,
    device=device
)
print(dataset_textile)

In [ ]:
dataset_textile.to_csv('dataset_textile.csv')

#### Генерация отзывов из категории __украшения и аксессуары__

In [ ]:
TARGET_COUNT = 500
CATEGORIES_TO_PROCESS = ["украшения и аксессуары"]
dataset_decorations = generate_dataset(
    target_total_count=TARGET_COUNT,
    full_clothing_df=X_train_clothes,
    categories_to_generate=CATEGORIES_TO_PROCESS,
    product_storage=PRODUCT_NAME_STORAGE,
    prompt_parts=PROMPT_PARTS,
    model=model,
    tokenizer=tokenizer,
    device=device
)
print(dataset_decorations)

In [ ]:
dataset_decorations.to_csv('dataset_decorations')

#### Генерация отзывов из категории __обувь__

In [ ]:
TARGET_COUNT = 500
CATEGORIES_TO_PROCESS = ["обувь"]
dataset_shoes = generate_dataset(
    target_total_count=TARGET_COUNT,
    full_clothing_df=X_train_clothes,
    categories_to_generate=CATEGORIES_TO_PROCESS,
    product_storage=PRODUCT_NAME_STORAGE,
    prompt_parts=PROMPT_PARTS,
    model=model,
    tokenizer=tokenizer,
    device=device
)
print(dataset_shoes)

In [ ]:
dataset_shoes.to_csv('dataset_shoes')

#### Генерация отзывов из категории __товары для детей__

In [ ]:
TARGET_COUNT = 500
CATEGORIES_TO_PROCESS = ["товары для детей"]
dataset_children = generate_dataset(
    target_total_count=TARGET_COUNT,
    full_clothing_df=X_train_clothes,
    categories_to_generate=CATEGORIES_TO_PROCESS,
    product_storage=PRODUCT_NAME_STORAGE,
    prompt_parts=PROMPT_PARTS,
    model=model,
    tokenizer=tokenizer,
    device=device
)
print(dataset_children)

In [ ]:
dataset_children.to_csv('dataset_children')

### Выводы: в целом получились неплохие синтетические данные, есть конечно к чему стремится, но боюсь времени уже нет
Ключевые проблемы:
* Достаточно много генерируется примеров где название товара ставится в начало